In [1]:
import pathlib
import numpy as np

from aiida import orm, load_profile
from aiida.engine import submit

from qe_tools import CONSTANTS

from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

# BaTiO3
Let's define the BTO structure to use as example for this tutorial.

In [19]:
cell = [
    [5.34198, -3.08419, 4.38073],
    [0.00000,  6.16839, 4.38073],
    [-5.3419, -3.08419, 4.38073],
]
cell = CONSTANTS.bohr_to_ang*np.array(cell)

structure = orm.StructureData(cell=cell)

structure.append_atom(position=np.dot([0.48674, 0.4867, 0.4867], cell),  symbols='Ti')
structure.append_atom(position=np.dot([0.51163, 0.5116, 0.0192], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.51163, 0.0192, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.01929, 0.5116, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.99899, 0.9989, 0.9989], cell),  symbols='Ba')

## HubbardStructureData initialization
Let's initialize the HubbardStructureData with the BTO structure!

> Important: You only need to do these steps once! Later just keep track of this testing structure via the PK and reload it from the database so you don't get 100 BaTiO3 strucures in your storage. ^^

In [3]:
hubbard_data = HubbardStructureData(structure=structure)

In [4]:
hubbard_data

<HubbardStructureData: uuid: 67266c48-354b-4c4d-ab46-de75da7fb284 (unstored)>

## Initializing the on-site Hubbard
Let's initialize the on-site Hubbard parameter for the titanium atom.

In [5]:
hubbard_data.initialize_onsites_hubbard('Ti', '3d', 5.0) # index 0 for Ti ==> Hubbard U

Here how it is stored in the class.

In [6]:
hubbard_data.hubbard_parameters

[[0, '3d', 0, '3d', 5.0, [0, 0, 0], 'dudarev']]

## Initializing the inter-site Hubbard
Let's initialize the inter-site Hubbard parameter for the titanium and oxygen atoms.

In [7]:
hubbard_data.initialize_intersites_hubbard('Ti', '3d', 'O', '2p', 0.0001, number_of_neighbours=7)

The parameters are saved in the property `hubbard_parameters` as a list.

In [8]:
hubbard_data.hubbard_parameters

[[0, '3d', 0, '3d', 5.0, [0, 0, 0], 'dudarev'],
 [0, '3d', 3, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 1, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 2, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 1, '2p', 0.0001, [0, 0, 1], 'dudarev'],
 [0, '3d', 2, '2p', 0.0001, [0, 1, 0], 'dudarev'],
 [0, '3d', 3, '2p', 0.0001, [1, 0, 0], 'dudarev'],
 [0, '3d', 3, '2p', 0.0001, [0, 0, -1], 'dudarev']]

In [9]:
hubbard_data.store()

<HubbardStructureData: uuid: 67266c48-354b-4c4d-ab46-de75da7fb284 (pk: 2184)>

In [10]:
hubbard_data.pk

2184

## Running the `SelfConsistentHubbardWorkChain`

In [20]:
hubbard_data = orm.load_node(2184)  # I load the node from the database instead of always regenerating it
pw_code = orm.load_code('qe-dev-pw@lumi-small')
hp_code = orm.load_code('qe-dev-hp@lumi-small')

builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='fast',
    overrides=pathlib.Path('hubbard_overrides.yaml')
)

builder.skip_first_relax = True

In [11]:
submit_node = submit(builder)
print(submit_node)

uuid: 0fee01c2-5af0-431d-9c50-57020aeea2ef (pk: 2248) (aiida.workflows:quantumespresso.hp.hubbard)


In [2]:
%%bash
verdi daemon restart


Restarting the daemon... OK


In [3]:
%%bash
verdi daemon status
verdi process list -a -p 1
verdi process show 2234
verdi process report 2234
verdi process status 2234

Profile: jgeiger
Daemon is running as PID 312006 since 2023-02-14 12:05:46
Active workers [4]:
    PID    MEM %    CPU %  started
-------  -------  -------  -------------------
2070240    0.157        0  2023-02-16 14:46:13
2070241    0.157        0  2023-02-16 14:46:13
2070242    0.157        0  2023-02-16 14:46:13
2070243    0.158        0  2023-02-16 14:46:13
Use verdi daemon [incr | decr] [num] to increase / decrease the amount of workers
  PK  Created    Process label                   Process State     Process status
----  ---------  ------------------------------  ----------------  ----------------
2209  4h ago     SelfConsistentHubbardWorkChain  ⨯ Excepted
2234  4h ago     SelfConsistentHubbardWorkChain  ⏹ Finished [402]
2236  4h ago     PwBaseWorkChain                 ⏹ Finished [0]
2237  4h ago     create_kpoints_from_distance    ⏹ Finished [0]
2241  4h ago     PwCalculation                   ⏹ Finished [0]
2248  4h ago     SelfConsistentHubbardWorkChain  ⏹ Finished [402]
225

In [7]:
%%bash
verdi computer show lumi-small 
# verdi process report 2310

---------------------------  -------------------------------------------------------------------------------------------------
Label                        lumi-small
PK                           3
UUID                         ef38453e-4556-4ffe-9ffb-498047db9df4
Description                  LUMI - Supercomputer in the North - `small` partition (128 cores CPU; max 200jobs, 1day, 4 nodes)
Hostname                     lumi.csc.fi
Transport type               core.ssh
Scheduler type               core.slurm
Work directory               /scratch/project_465000106/jgeiger/aiida
Shebang                      #!/bin/bash
Mpirun command               srun -u -n {tot_num_mpiprocs}
Default #procs/machine       128
Default memory (kB)/machine
Prepend text                 #SBATCH --account=project_465000106
                             #SBATCH --partition=small
                             #SBATCH --mem=227328
Append text
---------------------------  ------------------------------------------------